In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pickle

# Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')

# Load dữ liệu
data = pd.read_csv('./train-data/FOREX_DATA.csv')
# Hàm tạo dataset dạng sliding window với target đã có sẵn (không dùng scaler)
def create_multifeature_dataset(df):
    X = df[columns].values
    Y = df['target'].values
    return X, Y


# time_step = 1
X, y = create_multifeature_dataset(data)

print(X, y)

# Chia dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình LSTM cho bài toán phân loại nhị phân
model = Sequential([
    Input(shape=(time_step, len(columns))),
    LSTM(300, return_sequences=True),
    LSTM(300, return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Dùng hàm kích hoạt sigmoid cho đầu ra nhị phân
])

# Compile model với loss cho phân loại nhị phân
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Định nghĩa callback dừng sớm nếu val_loss không cải thiện
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Huấn luyện mô hình
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          batch_size=64, verbose=1, epochs=1000, callbacks=[early_stopping])

# Lưu model (không lưu scaler vì không sử dụng)
model.save('./model/predict_model_XAUUSD.h5')


[[-0.45   3.08   3.38  ...  0.44   0.64   0.51 ]
 [-3.48  -0.45   3.08  ...  2.09   0.44   0.64 ]
 [ 4.87  -3.48  -0.45  ...  1.74   2.09   0.44 ]
 ...
 [ 0.2    0.01   0.185 ...  0.115  0.     0.095]
 [ 0.015  0.2    0.01  ...  0.045  0.115  0.   ]
 [ 0.365  0.015  0.2   ...  0.     0.045  0.115]] [1 1 1 ... 0 0 0]
